In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import re


In [ ]:
with open("case_2_data_for_members.json", "r") as read_file:
    data = json.load(read_file)

with open("case_2_reference_without_resume_sorted.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
def get_overlap(str1, str2):
  s1 = set(str1.split())
  s2 = set(str2.split())
  return len(s1 & s2) / max(len(s2), 1)
# set(df.iloc[0,:]['candidate_description'].split())

In [ ]:
with open("case_2_data_for_members.json", "r") as read_file:
    data = json.load(read_file)

with open("case_2_reference_without_resume_sorted.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
import datetime

def add_person(df, resume, vacancy, target):
    for field in df.columns:
        if field not in resume:
            resume[field] = None
    if 'languageItem' in resume and resume['languageItem'] is not None and resume['languageItems'] is None:
        resume['languageItems'] =  resume['languageItems']


# education
    resume['ed_level'] = ""
    if 'educationItem' in resume:
      resume['edu_num'] = len(resume['educationItem']) if resume['educationItem'] is not None else 0
      for edu in resume['educationItem']:
          if edu['education_level'] is not None:
            resume['ed_level'] += edu['education_level'] + " "

          if edu['faculty'] is not None:
             edu['faculty'] =   re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', edu['faculty']).lower()
             if 'master' in edu['faculty'] or 'магистр' in edu['faculty']:
                resume['ed_level'] += "магистр "
          if edu['specialty'] is not None:
             edu['specialty'] =   re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', edu['specialty']).lower()
             if 'master' in edu['specialty'] or 'магистр' in edu['specialty']:
                resume['ed_level'] += "магистр "
          if edu['result'] is not None:
             edu['result'] =   re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', edu['result']).lower()
             if 'master' in edu['result'] or 'магистр' in edu['result']:
                resume['ed_level'] += "магистр "

    else:
      resume['edu_num'] = 0
    resume['ed_level'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', resume['ed_level']).lower()

    if resume['ed_level'] == "" or "неоконченное высшее" in resume['ed_level']:
        resume['ed_num'] = 0
    elif 'высшее' in  resume['ed_level'] or 'бакалавр' in resume['ed_level']:
        resume['ed_num'] = 1
    if 'магистр' in resume['ed_level'] or 'Master' in resume['ed_level']:
        resume['ed_num'] = 2

    resume['country'] = 1 if resume['country'] == 'Россия' or resume['country'] is None else 0



    # work experience
    resume['is_working'] = 0
    resume['work_cnt'] = 0

    if 'experienceItem' in resume and resume['experienceItem'] is not None:
        resume['work_cnt'] = len(resume['experienceItem'])
        descriptions = ""
        works = []
        resume['mean_work_dur'] = 0
        mwd_count = 0
        for item in resume['experienceItem']:
            if item['ends'] is None:
                resume['is_working'] = 1
            if item['description'] is not None:
                descriptions += " " + item['description']
                works.append(item['description'])
            if item['starts'] is not None and item['ends'] is not None:
                resume['mean_work_dur'] += (datetime.datetime.strptime(item['ends'], "%Y-%m-%d").date() - datetime.datetime.strptime(item['starts'], "%Y-%m-%d").date()).days
                mwd_count += 1
        resume['works_descriptions'] = descriptions
        resume['works_descriptions_arr'] = works
        if mwd_count != 0:
            resume['mean_work_dur'] /= mwd_count

    # texts
    #  vac_desc = (vacancy['comment'] or "") + " " + (vacancy['description'] or "") + " " + (vacancy['name'] or "") + (vacancy['keywords'] or "")

    resume['vac_keywords'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', vacancy["keywords"]).lower()
    resume["key_skills"] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', (resume["key_skills"] or "")).lower()

    vac_desc = (vacancy['comment'] or "") + " " + (vacancy['description'] or "") + " " + (vacancy['name'] or "")

    resume['vec_description'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', vac_desc).lower()

    candidate_descriprion = (resume["about"] or "") + " " + (resume["city"] or "") + " "
    candidate_descriprion += (resume["key_skills"] or "") + " " + (resume['works_descriptions'] or "")

    resume['candidate_description'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', candidate_descriprion).lower()

    resume['vac_uuid'] = vacancy["uuid"]
    resume['target'] = target

    df.loc[len(df.index)] = resume


df = pd.DataFrame(columns=['about', 'birth_date', 'city', 'country',
                          #  'educationItem', 'experienceItem',
                          'first_name', 'key_skills','languageItems',
                          'last_name',
                           'edu_num', 'ed_level', 'ed_num',
                           'work_cnt', 'is_working', 'mean_work_dur',
                           'works_descriptions',  'works_descriptions_arr',
                           'candidate_description',
                           'vac_description', 'vac_keywords',
                           'vac_uuid',
                           'target'])


# for vac in data:
#     for i in range(len(vac['failed_resumes'])):
#         add_person(df, vac['failed_resumes'][i], vac['vacancy'], 0)
#     for i in range(len(vac['confirmed_resumes'])):
#         add_person(df, vac['confirmed_resumes'][i], vac['vacancy'], 1)




TypeError: expected string or bytes-like object

In [ ]:
df['sex'] = df['first_name'].apply(lambda x: 0 if str(x).endswith("а") or str(x).endswith("я") else 1)

In [ ]:
df2 = pd.read_csv("auc-0.84.csv")

In [ ]:
def add_person_test(df, resume, vacancy):
    for field in df.columns:
        if field not in resume:
            resume[field] = None
    if 'languageItem' in resume and resume['languageItem'] is not None and resume['languageItems'] is None:
        resume['languageItems'] =  resume['languageItems']

# education
    resume['ed_level'] = ""
    if 'educationItem' in resume:
      resume['edu_num'] = len(resume['educationItem']) if resume['educationItem'] is not None else 0
      for edu in resume['educationItem']:
          if edu['education_level'] is not None:
            resume['ed_level'] += edu['education_level'] + " "

          if edu['faculty'] is not None:
             edu['faculty'] =   re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', edu['faculty']).lower()
             if 'master' in edu['faculty'] or 'магистр' in edu['faculty']:
                resume['ed_level'] += "магистр "
          if edu['specialty'] is not None:
             edu['specialty'] =   re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', edu['specialty']).lower()
             if 'master' in edu['specialty'] or 'магистр' in edu['specialty']:
                resume['ed_level'] += "магистр "
          if edu['result'] is not None:
             edu['result'] =   re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', edu['result']).lower()
             if 'master' in edu['result'] or 'магистр' in edu['result']:
                resume['ed_level'] += "магистр "

    else:
      resume['edu_num'] = 0
    resume['ed_level'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', resume['ed_level']).lower()

    if resume['ed_level'] == "" or "неоконченное высшее" in resume['ed_level']:
        resume['ed_num'] = 0
    elif 'высшее' in  resume['ed_level'] or 'бакалавр' in resume['ed_level']:
        resume['ed_num'] = 1
    if 'магистр' in resume['ed_level'] or 'Master' in resume['ed_level']:
        resume['ed_num'] = 2

    resume['country'] = 1 if resume['country'] == 'Россия' or resume['country'] is None else 0



    # work experience
    resume['is_working'] = 0
    resume['work_cnt'] = 0

    if 'experienceItem' in resume and resume['experienceItem'] is not None:
        resume['work_cnt'] = len(resume['experienceItem'])
        descriptions = ""
        works = []
        resume['mean_work_dur'] = 0
        mwd_count = 0
        for item in resume['experienceItem']:
            if item['ends'] is None:
                resume['is_working'] = 1
            if item['description'] is not None:
                descriptions += " " + item['description']
                works.append(item['description'])
            if item['starts'] is not None and item['ends'] is not None:
                resume['mean_work_dur'] += (datetime.datetime.strptime(item['ends'], "%Y-%m-%d").date() - datetime.datetime.strptime(item['starts'], "%Y-%m-%d").date()).days
                mwd_count += 1
        resume['works_descriptions'] = descriptions
        resume['works_descriptions_arr'] = works
        if mwd_count != 0:
            resume['mean_work_dur'] /= mwd_count

    # texts
    #  vac_desc = (vacancy['comment'] or "") + " " + (vacancy['description'] or "") + " " + (vacancy['name'] or "") + (vacancy['keywords'] or "")

    resume['vac_keywords'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', vacancy["keywords"]).lower()
    resume["key_skills"] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', (resume["key_skills"] or "")).lower()

    vac_desc = (vacancy['comment'] or "") + " " + (vacancy['description'] or "") + " " + (vacancy['name'] or "")

    resume['vec_description'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', vac_desc).lower()

    candidate_descriprion = (resume["about"] or "") + " " + (resume["city"] or "") + " "
    candidate_descriprion += (resume["key_skills"] or "") + " " + (resume['works_descriptions'] or "")

    resume['candidate_description'] = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', candidate_descriprion).lower()

    resume['vac_uuid'] = vacancy["uuid"]

    df.loc[len(df.index)] = resume


test_df = pd.DataFrame(columns=['about', 'birth_date', 'city', 'country',
                          #  'educationItem', 'experienceItem',
                          'first_name', 'key_skills','languageItems',
                          'last_name',
                           'edu_num', 'ed_level', 'ed_num',
                           'work_cnt', 'is_working', 'mean_work_dur',
                           'works_descriptions',  'works_descriptions_arr',
                           'candidate_description',
                           'vac_description', 'vac_keywords',
                           'vac_uuid',
                            'uuid'])

for i in range(len(test['resumes'])):
    add_person_test(test_df, test['resumes'][i], vac['vacancy'])

In [ ]:
test_df

,about,birth_date,city,country,first_name,key_skills,languageItems,last_name,edu_num,ed_level,...,work_cnt,is_working,mean_work_dur,works_descriptions,works_descriptions_arr,candidate_description,vac_description,vac_keywords,vac_uuid,uuid
0,"Личные качества: ответственность, лидерство, г...",1991-05-23,Москва,1,Клим,java core spring framework hibernate orm postg...,"[Русский, Английский]",Тетерина,1,высшее,...,5,1,274.000000,"Руководил командой разработчиков, в том числе...","[Руководил командой разработчиков, в том числе...",личные качества ответственность лидерство гибк...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33
1,Обязанности: в составе группы поддержки я отв...,1990-07-18,Нижний Новгород,1,Алиса,java 8 17 java se java ee spring framework boo...,None,Ситникова,0,,...,4,1,792.333333,Проект по поддержке инфраструктуры информаци...,[ Проект по поддержке инфраструктуры информаци...,обязанности в составе группы поддержки я отве...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,f8b69e24-e2c0-3186-9578-380835eb2ee7
2,Обучаемость Коммуникабельность Организованнос...,1990-01-01,Санкт-Петербург,1,Розалина,java 8 и 11 osgi postgresql testng mockito mav...,"[Русский, Английский]",Андреев,3,высшее высшее высшее,...,3,0,608.666667,"Анализ целевых систем: SAP (MDM, HCM, HANA, ...","[ Анализ целевых систем: SAP (MDM, HCM, HANA, ...",обучаемость коммуникабельность организованнос...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,e3976e74-e71b-34db-8e98-08dc422fa567
3,Общий опыт разработки на различных языках про...,1990-01-01,Москва,1,Антон,java ee spring framework intellij idea работа ...,"[Русский, Английский]",Кудрявцева,1,бакалавр,...,6,1,444.000000,Поддержка и развитие интеграционной шины дан...,[ Поддержка и развитие интеграционной шины дан...,общий опыт разработки на различных языках про...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,9a9c3ff1-49f8-30dd-a294-e56fc60cae64
4,"О себе: Java/Kotlin-разработчик, знаю Spring ...",1995-01-01,Москва,1,Александра,,"[Русский, Английский]",Панова,1,высшее,...,6,1,548.200000,"Some common tasks with PostgreSQL, Spring Bo...","[ Some common tasks with PostgreSQL, Spring Bo...",о себе java kotlin разработчик знаю spring bo...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,6561771c-7ef3-3e50-ab3a-ba8547201480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,None,1995-06-17,Москва,1,Кристина,,"[Английский, Русский]",Яковлева,1,высшее,...,2,0,623.000000,BIS. Корпоративная сервисная шина для создан...,[ BIS. Корпоративная сервисная шина для создан...,москва bis корпоративная сервисная шина для с...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,82df355a-235e-3046-9e6e-782ddf1600eb
109,Более 10 лет разработки приложений Windows и ...,1987-01-01,Мураши,1,Аполлон,java servlets hibernate orm soap rest spring b...,"[Русский, Английский]",Белякова,1,бакалавр,...,9,1,346.125000,Разработка и поддержка внутренних проектов к...,[ Разработка и поддержка внутренних проектов к...,более 10 лет разработки приложений windows и ...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,915597ce-24e5-31fa-8dca-29437f49f839
110,В свободное время люблю путешествовать с семьё...,1984-11-22,Москва,1,Клавдия,ответственность коммуникабельность аналитическ...,None,Пономарёв,2,высшее высшее,...,3,1,609.000000,Разработка программного продукта сбора и пров...,[Разработка программного продукта сбора и пров...,в свободное время люблю путешествовать с семь ...,None,vue js nuxt js vuex pinia html5 css scss less ...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,f288a532-0b58-30cb-ac3c-f87e53984719
111,Поиск нового места работы всегда был довольно ...,1993-06

In [ ]:
trans_proba = pd.read_csv("df_with_trans_test.csv")

In [ ]:
trans_proba['transformer_proba']

0      3.698055
1      3.261406
2      2.221629
3      4.289658
4      5.318633
         ...   
108    1.603039
109    6.396557
110    2.372243
111    2.956308
112    1.529585
Name: transformer_proba, Length: 113, dtype: float64

In [ ]:
test_df['new_transformer_proba'] = trans_proba['transformer_proba']

In [ ]:
%pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from lightgbm import LGBMRegressor, LGBMClassifier
import xgboost as XGB
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

features = ["work_age", "edu_num", "work_cnt", "is_working", "know_english", "language_count", "country", "age", "new_transformer_proba", "sex", "skills_count"]

X = df2[features]
y = df2["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

model = CatBoostClassifier(class_weights=[0.4, 0.6], eval_metric="Precision", depth=4, iterations=500, learning_rate=0.05)
# model = LGBMClassifier(class_weight={1: 0.4, 0:0.6}, objective='binary', metric='precision', max_depth=7, n_estimators=500, learning_rate=0.05)

model.fit(X_train, y_train)

prediction = model.predict(X_val)
proba =  model.predict_proba(X_val)[:, 1]

accuracy = (prediction == y_val).mean()
print(f'Accuracy on validation set: {accuracy}')

print(prediction)

print("Precision: ", precision_score(y_val, prediction))
print("Recall: ", recall_score(y_val, prediction))
print("PR score: ", average_precision_score(y_val, proba))
print("Auc score: ", roc_auc_score(y_val, proba))

0:	learn: 0.7039711	total: 1.54ms	remaining: 769ms
1:	learn: 0.7039711	total: 2.41ms	remaining: 600ms
2:	learn: 0.6574074	total: 3.21ms	remaining: 532ms
3:	learn: 0.6574074	total: 4.38ms	remaining: 543ms
4:	learn: 0.6614907	total: 5.67ms	remaining: 561ms
5:	learn: 0.6771005	total: 6.41ms	remaining: 528ms
6:	learn: 0.6771005	total: 7.16ms	remaining: 504ms
7:	learn: 0.6771005	total: 8.15ms	remaining: 501ms
8:	learn: 0.6771005	total: 9.14ms	remaining: 499ms
9:	learn: 0.6698113	total: 10.1ms	remaining: 496ms
10:	learn: 0.6805112	total: 11.1ms	remaining: 493ms
11:	learn: 0.6945813	total: 12.1ms	remaining: 490ms
12:	learn: 0.7079796	total: 13ms	remaining: 488ms
13:	learn: 0.7061770	total: 13.8ms	remaining: 481ms
14:	learn: 0.7061770	total: 14.8ms	remaining: 479ms
15:	learn: 0.7167236	total: 15.8ms	remaining: 477ms
16:	learn: 0.7152659	total: 16.8ms	remaining: 476ms
17:	learn: 0.7109244	total: 17.9ms	remaining: 479ms
18:	learn: 0.7046980	total: 18.9ms	remaining: 478ms
19:	learn: 0.7070707	tot

In [ ]:
test_df["birth_date"] = pd.to_datetime(test_df["birth_date"])

def count_skills(langs):
    if type(langs) != str:
      return 0

    return langs.count(' ') + 1

test_df["skills_count"] = test_df["key_skills"].apply(count_skills)

In [ ]:
from datetime import datetime

def calculate_age(birth_date):
    today = datetime.today()
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age

test_df["age"] = test_df["birth_date"].apply(calculate_age)

test_df["work_age"] = test_df["work_cnt"] / test_df["age"]

In [ ]:
def count_languages(langs):
    if type(langs) != str:
      return 0

    return langs.count(',') + 1

test_df['sex'] = test_df['first_name'].apply(lambda x: 0 if str(x).endswith("а") or str(x).endswith("я") else 1)

test_df["language_count"] = test_df["languageItems"].apply(count_languages)

def check_english(langs):
    if type(langs) != str:
      return 0

    if "Английский" in langs:
        return 1
    else:
        return 0

test_df["know_english"] = test_df["languageItems"].apply(check_english)

In [ ]:
features = ["work_age", "edu_num", "work_cnt", "is_working", "know_english", "language_count", "country", "age", "new_transformer_proba", "sex", "skills_count"]

X_test = test_df[features]
prediction_proba = model.predict_proba(X_test)[:, 1]
prediction = model.predict(X_test)

In [ ]:
(prediction_proba > 0.75).sum()

54

In [ ]:
res = []
for i, el_id in enumerate(test_df['uuid']):
    if prediction_proba[i] > 0.75:
      res.append(el_id)

In [ ]:
res

['0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33',
 'f8b69e24-e2c0-3186-9578-380835eb2ee7',
 'e3976e74-e71b-34db-8e98-08dc422fa567',
 '6561771c-7ef3-3e50-ab3a-ba8547201480',
 '5ae3b7a0-6028-3bd8-8b4e-27a861503806',
 '8e456229-6e4e-3d57-a90e-ae4c3e1db869',
 'd587e742-f07f-3037-b412-ac12c89f6e6a',
 '6abae195-2cbf-3bbe-8dfb-8d2dcc3e88b6',
 '9fe9cded-6120-384e-8f9a-96ce33bc99dd',
 '9cef8747-af3a-3833-9098-3bfabf1e40eb',
 'c2a6bf5a-45ca-3f79-93ff-355a724b65df',
 '6065ee07-4c42-3cf4-8e45-9d053b9041f7',
 '42eae4bf-4826-3105-b197-dc0afb063714',
 'a71b0749-1ebc-3099-b0a1-342ca64d1575',
 'ae797830-f7a6-38f3-abf8-e1dc6cf3b6c3',
 '0cfa3d00-10b5-38d5-b166-329ff6f4598b',
 '5cfd26bc-84e6-3714-b54b-746acbca3d18',
 '7a79f64e-7a87-3008-b8ec-e165a94f72e0',
 'e58e684a-509e-33c6-b88c-6d3c07d348bb',
 'b1357f8e-3c98-3142-81e7-fd729ec50ca0',
 '099b1c65-c060-3f5d-87cf-42bda251c3ea',
 '3c4020e7-172f-3ed5-adc9-fe9a62097b26',
 'cc683ed5-8db5-32da-9eab-9433a539b3b5',
 'eb02a87d-d829-36bc-9006-11bceef0ffeb',
 'add58503-f251-

In [ ]:
with open('result.txt', 'w') as f:
    for el in res:
      f.write(el + '\n')